In [44]:
# 68 13 april random forest and 14 april:

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
import seaborn as sns
data = sns.load_dataset('tips')
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [47]:
data['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [48]:
data['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [49]:
data.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [50]:
# encoding:

In [51]:
from sklearn.preprocessing import LabelEncoder

In [52]:
encoder = LabelEncoder()

In [53]:
data['time']  = encoder.fit_transform(data['time'])

In [54]:
X = data.drop(labels=['time'], axis = 1)
y = data['time']

In [55]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [56]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state= 42)

In [59]:
X_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


In [60]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [61]:
# separating numerical and categorical variables

In [62]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195 entries, 228 to 102
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  195 non-null    float64 
 1   tip         195 non-null    float64 
 2   sex         195 non-null    category
 3   smoker      195 non-null    category
 4   day         195 non-null    category
 5   size        195 non-null    int64   
dtypes: category(3), float64(2), int64(1)
memory usage: 7.1 KB


In [63]:
cat_col = ['sex', 'smoker', 'day']
num_col = ['total_bill', 'tip', 'size']

In [64]:
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')), 
        ('scaler', StandardScaler())
    ]
)

In [65]:
cat_pipeline = Pipeline(
    steps= [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder())
    ]
)

In [66]:
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, num_col), 
    ('cat_pipeline', cat_pipeline, cat_col)
]
)

In [67]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [68]:
X_train[1]

array([ 0.46322744, -0.74211442, -0.61214068,  0.        ,  1.        ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        0.        ])

In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [70]:
models = {
    'random_forest': RandomForestClassifier(oob_score=True),
    'logistic_regression': LogisticRegression(),
    'decision_tree': DecisionTreeClassifier()
    }

In [71]:
from sklearn.metrics import accuracy_score

In [72]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)
        print(model.score(X_train,y_train))
        #print("OOB score:", model.oob_score_)
        
        y_pred=model.predict(X_test)
        
        accuracy=round(accuracy_score(y_test,y_pred)*100,2)
        
        report[list(models.keys())[i]]=accuracy
    
    return report

In [73]:
evaluate_model(X_train, y_train, X_test, y_test, models)

1.0
0.9538461538461539
1.0


{'random_forest': 95.92, 'logistic_regression': 100.0, 'decision_tree': 93.88}

# Reducing over-fitting with Random forest: 

In [74]:
params={
    
    "n_estimators":[50,100,200],
    "criterion":["gini","entropy"],
    "max_depth":[3,5,10]
    
}

In [75]:
model=RandomForestClassifier(oob_score=True)

In [76]:
from sklearn.model_selection import RandomizedSearchCV

In [77]:
cv=RandomizedSearchCV(model,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [78]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.974 total time=   0.6s


[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.923 total time=   0.7s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=1.000 total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.923 total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.923 total time=   0.4s
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.949 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.949 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=50;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(oob_score=True),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   scoring='accuracy', verbose=3)

In [79]:

cv.best_params_

{'n_estimators': 100, 'max_depth': 5, 'criterion': 'entropy'}

In [85]:
best_model=RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion='entropy',oob_score=True)

In [86]:
best_model.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=5, oob_score=True)

In [90]:
best_model.oob_score_ # out of bag samples are kept separately for model evaluation

0.9487179487179487

In [87]:
y_pred=best_model.predict(X_test)

In [88]:
best_accuracy=round(accuracy_score(y_test,y_pred)*100,2)

In [89]:
best_accuracy # we reduced over fitting to some extent...

97.96